In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
import glob

pos_file_names = glob.glob('./train/train/pos/*')
neg_file_names = glob.glob('./train/train/neg/*')

In [3]:
reviews = []
for fn in pos_file_names:
    with open(fn, encoding='utf-8') as f:
        content = f.read()
    reviews.append((content,1))
    
for fn in neg_file_names:
    with open(fn, encoding='utf-8') as f:
        content = f.read()
    reviews.append((content,0))

In [4]:
reviews_df = pd.DataFrame(reviews, columns=['review', 'sentiment'])

In [5]:
reviews_df

,review,sentiment
0,Bromwell High is a cartoon comedy. It ran at t...,1
1,Homelessness (or Houselessness as George Carli...,1
2,Brilliant over-acting by Lesley Ann Warren. Be...,1
3,This is easily the most underrated film inn th...,1
4,This is not the typical Mel Brooks film. It wa...,1
5,"This isn't the comedic Robin Williams, nor is ...",1
6,Yes its an art... to successfully make a slow ...,1
7,"In this ""critically acclaimed psychological th...",1
8,THE NIGHT LISTENER (2006) **1/2 Robin Williams...,1
9,"You know, Robin Williams, God bless him, is co...",1


In [6]:
from keras.preprocessing import text

C:\Users\Jared\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [7]:
vocab_size = 500
emb_size = 100

In [8]:
token = text.Tokenizer(num_words=vocab_size)

In [9]:
token.fit_on_texts(reviews_df.review)

In [10]:
review_seq = token.texts_to_sequences(reviews_df.review)

In [11]:
from keras.preprocessing.sequence import pad_sequences
review_seq = pad_sequences(review_seq)

In [12]:
from sklearn.cross_validation import train_test_split

C:\Users\Jared\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [13]:
X_train, X_val, y_train, y_val = train_test_split(review_seq, reviews_df['sentiment'], train_size=0.6, random_state=50) 
#random_state is 50 because 42 is basic

In [14]:
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, train_size=0.5, random_state=50)

In [15]:
from keras.models import Sequential
from keras.layers import CuDNNLSTM, CuDNNGRU, Dense, Flatten, Embedding, Convolution1D, TimeDistributed, MaxPool1D, LeakyReLU
from keras.callbacks import EarlyStopping
from keras.optimizers import RMSprop

In [16]:
model = Sequential()
model.add(Embedding(len(token.word_index) + 1, emb_size, input_length=pad_shape[1]))
model.add(TimeDistributed(Convolution1D(150, 5, input_length=pad_shape[1])))
model.add(TimeDistributed(LeakyReLU()))
model.add(TimeDistributed(MaxPool1D(pool_size=2)))
model.add(TimeDistributed(Flatten()))
model.add(CuDNNLSTM(100, return_sequences=True))
model.add(CuDNNLSTM(100, return_sequences=True))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

NameError: name 'pad_shape' is not defined

In [ ]:
rms = RMSprop()

In [ ]:
es = EarlyStopping(patience=2)

In [ ]:
model.compile(rms, 'mse', metrics=['accuracy'])
model.fit(X_train, np.array(y_train), callbacks=[es], validation_data=(X_val, y_val), epochs=50)